# Install dependencies


In [ ]:
!pip install persiantools
import json
from google.colab import files
import pandas as pd
import io
from persiantools.jdatetime import JalaliDate
import datetime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Upload Json file 


**Sample structure**
```
{
  "data": {
    "1400/01/01": "100",
    "1400/01/02": "150",
    "1400/01/03": "200",
    "1400/01/22": "800"
  },
  "config": {
    "type": "shamsi",
    "time": "daily",
    "interpolation": "linear"
  }
}
```



In [ ]:
#choose your json file to be uploaded
uploaded = files.upload()

Saving sample.json to sample.json


# Read each data and config from uploaded Json seprately into Pandas Dataframes





In [ ]:
# replace your file_name by your own file name. (ex: 'sample.json')
data = json.loads(uploaded['file_name'].decode("utf-8"))
input_time_series = pd.DataFrame(data["data"].items(), columns=['Date', 'price'])

config = pd.DataFrame(data["config"].items())
config.columns=['params', 'option']
config = config.set_index('params')
print(input_time_series)
print(config)

         Date price
0  1400/01/01   100
1  1400/01/02   150
2  1400/01/03   200
3  1400/01/22   800
               option
params               
type           shamsi
time            daily
interpolation  linear


# To simplify the process we convert the Shamsi date to gregorian (if needed)

In [ ]:
# function that get the data and convert
def to_gregorian_date(x):
  year, month, day=x.split('/', 3)

  return JalaliDate(int(year), int(month), int(day)).to_gregorian()

In [ ]:
date_type =  config.iloc[config.index=='type']['option'].values[0]
ts=input_time_series
if date_type == 'shamsi':
  ts['Date'] = ts['Date'].apply(to_gregorian_date)
ts['Date'] = ts['Date'].astype('datetime64[ns]')
data = ts.sort_values(by=['Date'], ascending=[True])
ts = ts.set_index('Date')
ts['price']= ts['price'].astype('int')
print(ts)

            price
Date             
2021-03-21    100
2021-03-22    150
2021-03-23    200
2021-04-11    800


# Generate the missing dates based on defined frequency

In [ ]:
time_type =  config.iloc[config.index=='time']['option'].values[0]

if time_type == 'daily':
  time_type = 'd'
elif time_type == 'monthly':
  time_type = 'm'
elif time_type == 'yearly':
  time_type = 'y'

ts = ts.asfreq(time_type)
print(ts)

            price
Date             
2021-03-21  100.0
2021-03-22  150.0
2021-03-23  200.0
2021-03-24    NaN
2021-03-25    NaN
2021-03-26    NaN
2021-03-27    NaN
2021-03-28    NaN
2021-03-29    NaN
2021-03-30    NaN
2021-03-31    NaN
2021-04-01    NaN
2021-04-02    NaN
2021-04-03    NaN
2021-04-04    NaN
2021-04-05    NaN
2021-04-06    NaN
2021-04-07    NaN
2021-04-08    NaN
2021-04-09    NaN
2021-04-10    NaN
2021-04-11  800.0


1. *Interpolation*

# Interpolate the missing values 
(Bonus task) Diffrent type of interpolation can be run

In [ ]:
interpolation_type =  config.iloc[config.index=='interpolation']['option'].values[0]
print(interpolation_type)
def interpolation_f(input_data, Interpolation_type):
  if interpolation_type == 'linear':
    return input_data.interpolate(method='linear')
  elif interpolation_type == 'spline':
    return input_data.interpolate(method='spline', order=3)
  elif interpolation_type == 'polynomial':
    return input_data.interpolate(method='polynomial', order=2)
  else:
    return input_data


ts['price'] = interpolation_f(ts['price'], interpolation_type)
print(ts)

linear
                 price
Date                  
2021-03-21  100.000000
2021-03-22  150.000000
2021-03-23  200.000000
2021-03-24  231.578947
2021-03-25  263.157895
2021-03-26  294.736842
2021-03-27  326.315789
2021-03-28  357.894737
2021-03-29  389.473684
2021-03-30  421.052632
2021-03-31  452.631579
2021-04-01  484.210526
2021-04-02  515.789474
2021-04-03  547.368421
2021-04-04  578.947368
2021-04-05  610.526316
2021-04-06  642.105263
2021-04-07  673.684211
2021-04-08  705.263158
2021-04-09  736.842105
2021-04-10  768.421053
2021-04-11  800.000000


# Return the miladi to Shami date if needed

In [ ]:
# function that convert miladi to shamsi
def to_jalaly_date(x):
  x = x.strftime("%Y-%m-%d")
  year, month, day=x.split('-', 3)

  return JalaliDate.to_jalali(int(year), int(month), int(day)).strftime("%Y/%m/%d")

In [ ]:
if date_type == 'shamsi':
  ts = ts.reset_index()
  ts['Date'] = ts['Date'].apply(to_jalaly_date)
  



# Print the output JSON

In [ ]:
ts.set_index('Date')['price'].to_dict()


{'1400/01/01': 100.0,
 '1400/01/02': 150.0,
 '1400/01/03': 200.0,
 '1400/01/04': 231.57894736842104,
 '1400/01/05': 263.1578947368421,
 '1400/01/06': 294.7368421052631,
 '1400/01/07': 326.3157894736842,
 '1400/01/08': 357.89473684210526,
 '1400/01/09': 389.4736842105263,
 '1400/01/10': 421.0526315789474,
 '1400/01/11': 452.63157894736844,
 '1400/01/12': 484.2105263157895,
 '1400/01/13': 515.7894736842105,
 '1400/01/14': 547.3684210526316,
 '1400/01/15': 578.9473684210526,
 '1400/01/16': 610.5263157894736,
 '1400/01/17': 642.1052631578948,
 '1400/01/18': 673.6842105263158,
 '1400/01/19': 705.2631578947369,
 '1400/01/20': 736.8421052631579,
 '1400/01/21': 768.421052631579,
 '1400/01/22': 800.0}

## Writing data into a file

In [ ]:
#we can also write the result into a Json file 
dictionary = ts.set_index('Date')['price'].to_dict()
with open('json_data.json', 'w') as outfile:
    json.dump(dictionary, outfile)